In [ ]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
import numpy as np
import logging

from ppp.splines import InoueBspline
from ppp.interpolation import PoissonProcess

client = MongoClient("129.26.78.40:27017")
db = client['BitCoin']
logging.basicConfig(level=logging.INFO)

In [ ]:
r = db['senders_2017_week_2'].aggregate([
                                        {"$match": {"ntx": {"$gte": 10}}},
                                        {"$unwind": "$tx"},
                                        {"$group": {
                                            "_id": {"s": "$_id", "ts": "$tx.ts" },                                        
                                        }},
                                        {"$group": {
                                            "_id": "$_id.s",
                                            "txs": {"$addToSet": "$_id.ts"},
                                            "ntx": {"$sum": 1}
                                        }},
                                        {"$match": {"ntx": {"$gte": 10}}}
                                    ],
                            allowDiskUse=True)

In [ ]:
r.next()

In [ ]:
senders = []
for d in r:
    #print d["_id"]
    txs = (np.asarray(d['txs'])  - 1483228800) / (24.*3600)
    senders.append(txs)
    

In [ ]:
len(senders)

In [ ]:
senders[0]

In [ ]:
model =  PoissonProcess("a", InoueBspline(M=7, min_arrival=0., max_arrival=7.,arrivals=senders[13002] % 7 ))
    
        
model.fit(20, .0001)


In [ ]:
model.plot()
plt.show()

In [ ]:
arrivals

In [ ]:
a = db['senders_2017_week_1'].aggregate([
                                        {"$match": {"ntx": {"$gte": 5}}},
                                        {"$unwind": "$tx"},
                                        {"$group": {
                                            "_id": {"s": "$_id", "ts": "$tx.ts", "t": "$tx.t"}
                                        }}
                                    ], allowDiskUse=True)

In [ ]:
a.next()

In [1]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
import numpy as np
import logging

client = MongoClient("129.26.78.40:27017")
client.database_names()
db = client["BitCoin"]


In [2]:
data = []
addresses = db['senders_2017_week_1'].aggregate([
                                        {"$match": {"ntx": {"$gte": 10}}},
                                        {"$unwind": "$tx"},
                                        {"$group": {
                                            "_id": {"s": "$_id", "ts": "$tx.ts" },
                                        }},
                                        {"$group": {
                                            "_id": "$_id.s",
                                            "txs": {"$addToSet": "$_id.ts"},
                                            "ntx": {"$sum": 1}
                                        }},
                                        {"$match": {"ntx": {"$gte": 10}}}
                                    ],
                            allowDiskUse=True)
for address in addresses:
    txs = (np.asarray(address['txs'])  - 1483228800) / (24.*3600)
    data.append((address["_id"], txs % 7))


In [4]:
len(data)

7185